In [24]:
import os
from pyspark import SparkContext
from pyspark.ml.linalg import Vector, Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.session import SparkSession

# pipeline 模式
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer, ChiSqSelector, IDF, IndexToString, StringIndexer, VectorIndexer

os.environ['JAVA_HOME']
sc = SparkContext( 'local', 'test')
spark = SparkSession(sc)

'/java/jdk1.8'

In [69]:
# help(sc.stop)
sc.stop()

In [7]:
textFile = sc.textFile("./word.txt")
textFile
textFile.first()
# textFile.saveAsTextFile('./writeback')  # 保存新文件到writeback路径
# logData = sc.textFile(logFile, 2).cache()
# numAs = logData.filter(lambda line: 'a' in line).count()
# numBs = logData.filter(lambda line: 'b' in line).count()
# print('Lines with a: %s, Lines with b: %s' % (numAs, numBs))

./word.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

'asjdklsadasjdas;ldf'

In [5]:
# 导入向量和模型
# from pyspark.ml.linalg import Vectors
# from pyspark.ml.classification import LogisticRegression
# from pyspark.sql.session import SparkSession

# 准备训练数据
# Prepare training data from a list of (label, features) tuples
# spark = SparkSession(sc)
training = spark.createDataFrame([
(1.0,Vectors.dense([0.0,1.1,0.1])),
(0.0,Vectors.dense([2.0,1.0,-1.0])),
(0.0,Vectors.dense([2.0,1.3,1.0])),
(1.0,Vectors.dense([0.0,1.2,-0.5]))],["label","features"])

# 创建回归实例，这个实例是Estimator
# Create a LogisticRegression instance.This instance is an Estimator.
lr = LogisticRegression(maxIter=10, regParam=0.01)
# print("LogisticRegression parameters:\n" + lr.explainParams() + '\n')

# 使用lr中的参数训练出model
# Learn a LogisticRegression model.This uses the parameters stored in lr.
model1 = lr.fit(training)

# Since model1 is a Model (i.e., a transformer produced by an Estimator),
# we can view the parameters it used during fit().
# This prints the parameter (name: value) pairs, where names are unique IDs for this
# LogisticRegression instance.

#查看model1在fit()中使用的参数
# print("Model 1 was fit using parameters: ")
# print(model1.extractParamMap())

paramMap = {lr.maxIter: 20}
paramMap[lr.maxIter] = 30
paramMap.update({lr.regParam: 0.1, lr.threshold: 0.55})

# You can combine paramMaps, which are python dictionaries.
# 新的参数，合并为两组参数对
paramMap2 = {lr.probabilityCol: 'myProbability'}
paramMapCombined = paramMap.copy()
paramMapCombined.update(paramMap2)
# paramMapCombined

# 重新得到model2并拿出来参数瞅瞅
model2 = lr.fit(training, paramMapCombined)
# print('Model 2 was fit using parameters: ')
# print(model2.extractParamMap())

# 准备测试的数据
# Prepare test data
test = spark.createDataFrame([
(1.0,Vectors.dense([-1.0,1.5,1.3])),
(0.0,Vectors.dense([3.0,2.0,-0.1])),
(1.0,Vectors.dense([0.0,2.2,-1.5]))],["label","features"])

prediction = model2.transform(test)
selected = prediction.select('features', 'label', 'myProbability', 'prediction')
for row in selected.collect():
    print(row)

Row(features=DenseVector([-1.0, 1.5, 1.3]), label=1.0, myProbability=DenseVector([0.0571, 0.9429]), prediction=1.0)
Row(features=DenseVector([3.0, 2.0, -0.1]), label=0.0, myProbability=DenseVector([0.9239, 0.0761]), prediction=0.0)
Row(features=DenseVector([0.0, 2.2, -1.5]), label=1.0, myProbability=DenseVector([0.1097, 0.8903]), prediction=1.0)


In [24]:
help(LogisticRegression)

SyntaxError: invalid syntax (<ipython-input-24-605d976ba39d>, line 2)

In [6]:
#准备测试数据
# Pipeline 跑
# Prepare training documents from a list of (id, text, label) tuples.
training = spark.createDataFrame([
(0,"a b c d e spark",1.0),
(1,"b d",0.0),
(2,"spark f g h",1.0),
(3,"hadoop mapreduce",0.0)],["id","text","label"])
#构建机器学习流水线
# Configure an ML pipeline, which consists of three stages: tokenizer, hashingTF, and lr.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.01)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])
print('What pipeline is : \n', pipeline.params)



What pipeline is : 
 [Param(parent='Pipeline_37e203d3d025', name='stages', doc='a list of pipeline stages')]


In [ ]:
#训练出model 此处霸占太多资源无法跑通！！！！
# Fit the pipeline to training documents.
model = pipeline.fit(training)
print('What model is :\n', model)


In [ ]:
#测试数据
# Prepare test documents, which are unlabeled (id, text) tuples.
test = spark.createDataFrame([
(4,"spark i j k"),
(5,"l m n"),
(6,"mapreduce spark"),
(7,"apache hadoop")],["id","text"])

#预测，打印出想要的结果
# Make predictions on test documents and print columns of interest.
prediction = model.transform(test)
selected = prediction.select("id","text","prediction")

for row in selected.collect():
    print(row)

In [25]:
help(HashingTF)


Help on class HashingTF in module pyspark.ml.feature:

class HashingTF(pyspark.ml.wrapper.JavaTransformer, pyspark.ml.param.shared.HasInputCol, pyspark.ml.param.shared.HasOutputCol, pyspark.ml.param.shared.HasNumFeatures, pyspark.ml.util.JavaMLReadable, pyspark.ml.util.JavaMLWritable)
 |  Maps a sequence of terms to their term frequencies using the hashing trick.
 |  Currently we use Austin Appleby's MurmurHash 3 algorithm (MurmurHash3_x86_32)
 |  to calculate the hash code value for the term object.
 |  Since a simple modulo is used to transform the hash function to a column index,
 |  it is advisable to use a power of two as the numFeatures parameter;
 |  otherwise the features will not be mapped evenly to the columns.
 |  
 |  >>> df = spark.createDataFrame([(["a", "b", "c"],)], ["words"])
 |  >>> hashingTF = HashingTF(numFeatures=10, inputCol="words", outputCol="features")
 |  >>> hashingTF.transform(df).head().features
 |  SparseVector(10, {0: 1.0, 1: 1.0, 2: 1.0})
 |  >>> hashing

In [67]:
# TF-IDF 特征抽取
sentenceData = spark.createDataFrame([
    (0, "I heard about Spark and I love Spark"),
    (0, "I wish Java could use case classes"),
    (1, "Logistic regression models are neat")
]).toDF("label", "sentence")
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
# help(tokenizer.transform)
wordsData = tokenizer.transform(sentenceData)
# help(wordsData.show)
sorted(list(wordsData.select('words').toPandas()['words'])[0], reverse=True)
wordsData.show(truncate=False)

# 用HashingTF把句子哈希成特征向量，设置哈希表桶数为2000
# 分词序列被变换成一个稀疏特征向量，其中每个单词都被散列成了一个不同的索引值，
# 特征向量在某一维度上的值即该词汇在文档中出现的次数。
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=2000)
featurizedData = hashingTF.transform(wordsData)
featurizedData.select('rawFeatures').show(truncate=False)

# 使用IDF对单纯的词频特征向量进行修正
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)
rescaledData.select('features', 'label').show(truncate=False)

['spark', 'spark', 'love', 'i', 'i', 'heard', 'and', 'about']

+-----+------------------------------------+---------------------------------------------+
|label|sentence                            |words                                        |
+-----+------------------------------------+---------------------------------------------+
|0    |I heard about Spark and I love Spark|[i, heard, about, spark, and, i, love, spark]|
|0    |I wish Java could use case classes  |[i, wish, java, could, use, case, classes]   |
|1    |Logistic regression models are neat |[logistic, regression, models, are, neat]    |
+-----+------------------------------------+---------------------------------------------+

+---------------------------------------------------------------------+
|rawFeatures                                                          |
+---------------------------------------------------------------------+
|(2000,[240,333,1105,1329,1357,1777],[1.0,1.0,2.0,2.0,1.0,1.0])       |
|(2000,[213,342,489,495,1329,1809,1967],[1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|(

In [21]:
# 卡方校验
df = spark.createDataFrame([
    (7, Vectors.dense([0.0, 0.0, 18.0, 1.0]), 1.0,),
    (8, Vectors.dense([0.0, 1.0, 12.0, 0.0]), 0.0,),
    (9, Vectors.dense([1.0, 0.0, 15.0, 0.1]), 0.0,)], ["id", "features", "clicked"])
selector = ChiSqSelector(numTopFeatures=1, featuresCol="features", outputCol="selectedFeatures", labelCol="clicked")
result = selector.fit(df).transform(df)
result.show()

df = spark.createDataFrame([
    (7, Vectors.dense([10.0, 3.3, 18.0, 7.0]), 1.0,),
    (8, Vectors.dense([0.0, 1.0, 12.0, 5.0]), 0.0,),
    (9, Vectors.dense([1.0, 0.0, 15.0, 3.1]), 0.0,)], ["id", "features", "clicked"])
selector = ChiSqSelector(numTopFeatures=1, featuresCol="features", outputCol="selectedFeatures", labelCol="clicked")
result = selector.fit(df).transform(df)
result.show()

+---+------------------+-------+----------------+
| id|          features|clicked|selectedFeatures|
+---+------------------+-------+----------------+
|  7|[0.0,0.0,18.0,1.0]|    1.0|          [18.0]|
|  8|[0.0,1.0,12.0,0.0]|    0.0|          [12.0]|
|  9|[1.0,0.0,15.0,0.1]|    0.0|          [15.0]|
+---+------------------+-------+----------------+

+---+-------------------+-------+----------------+
| id|           features|clicked|selectedFeatures|
+---+-------------------+-------+----------------+
|  7|[10.0,3.3,18.0,7.0]|    1.0|          [10.0]|
|  8| [0.0,1.0,12.0,5.0]|    0.0|           [0.0]|
|  9| [1.0,0.0,15.0,3.1]|    0.0|           [1.0]|
+---+-------------------+-------+----------------+



In [ ]:
"""
决策树实现参考：
https://blog.csdn.net/baidu_41605403/article/details/83006973
kaggle参考：
https://www.kaggle.com/benhamner/python-data-visualizations/notebook
"""